Using CWT-SSIM, we explore an interesting approach to classification, very different from everything we have seen till this point.

In [1]:
from __future__ import division
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from ssim import SSIM
from ssim.utils import get_gaussian_kernel
from sklearn import datasets, svm, metrics, decomposition
import pandas as pd
digits = datasets.load_digits()
import math
from math import *
from __future__ import division
import sklearn
from sklearn.tree import DecisionTreeClassifier
import pywt
import seaborn as sns
sns.set()
from matplotlib import cm

from sklearn.model_selection import train_test_split

n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

In [2]:
#image_transform = [Image.fromarray(im.reshape(8,8)/np.max(im)) for im in data]
image_reshape = [im.reshape(8,8)/(np.max(im)) for im in data]

image_transform = [ImageOps.grayscale(Image.fromarray(np.uint8(cm.Greys_r(myarray)*255))) for myarray in image_reshape]

In [117]:
SSIM( image_transform[8]) .cw_ssim_value(image_transform[9]) #just checking if sufficient "difference" exists between classes

0.95470980114388304

<b>Idea 1</b> Let's procede naively. Set aside the first 10 images, and then calculate CWT-SSIM using those 10 as the "template".

In [118]:
digits.target[0:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [119]:
ssim_vals_result = []
for classless_image in image_transform[10::]:
    ssim_vals = []
    for classed_image in image_transform[0:10]:
        ssim_vals.append(SSIM(classed_image).cw_ssim_value(classless_image))
    ssim_vals_result.append(ssim_vals)

In [120]:
np.array(ssim_vals_result).shape, len(data)

((1787L, 10L), 1797)

In [121]:
labels_actual = digits.target[10::]
labels_predicted = np.array([np.argmax(x) for x in ssim_vals_result])

In [122]:
np.count_nonzero(labels_actual==labels_predicted)/len(labels_actual)

0.49524342473419136

Hmm. That may have been a little too naive.
How about running a "closest match" in a test set, based on the train set?

In [3]:
X_train, X_test, y_train, y_test = train_test_split(image_transform, digits.target, test_size=0.33, random_state=42)

In [10]:
np.append(1, [1,2,3,4])

array([1, 1, 2, 3, 4])

In [ ]:
#this takes forever to run---don't try.
# I have a parallel version that works really well that I've given out as a .py script

predicted_class = []

for im_1, indx in zip(X_test, range(len(X_test))):

    ssim_master = np.array([])
    
    for im_2 in X_train:
        ssim_master = np.append(ssim_master, SSIM(im_1).cw_ssim_value(im_2))
    
    predicted_class.append(y_train[np.argmax(ssim_master)])
    
    if indx%10==0:
        print "current accuracy (%s images): %.2f"%(indx+1,  np.count_nonzero(y_test[0:indx+1]==np.array(predicted_class))/(indx+1))

current accuracy (1 images): 1.00
current accuracy (11 images): 0.91
current accuracy (21 images): 0.95
current accuracy (31 images): 0.97
current accuracy (41 images): 0.98
current accuracy (51 images): 0.94
current accuracy (61 images): 0.95
current accuracy (71 images): 0.96
current accuracy (81 images): 0.95
current accuracy (91 images): 0.95
current accuracy (101 images): 0.94
current accuracy (111 images): 0.95
current accuracy (121 images): 0.94
current accuracy (131 images): 0.95
current accuracy (141 images): 0.95
current accuracy (151 images): 0.95
current accuracy (161 images): 0.96
current accuracy (171 images): 0.95
current accuracy (181 images): 0.96
current accuracy (191 images): 0.96
current accuracy (201 images): 0.95
current accuracy (211 images): 0.95
current accuracy (221 images): 0.95
current accuracy (231 images): 0.96
current accuracy (241 images): 0.95
current accuracy (251 images): 0.95
current accuracy (261 images): 0.95
current accuracy (271 images): 0.95
cur